In [1]:
import torch

/root/miniconda3/envs/gread/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1 实现kernel中每个位置为两个节点特征之差
实现kernel中的$e_{i,j}= x_i - x_j$, $E=(e_{ij})$ 

In [17]:
n=8
d=3
x = torch.eye(n,d)
x_expanded = x.unsqueeze(0).repeat(n, 1, 1)
print(x.shape, '\n', x_expanded.shape)
diff = x_expanded - x.unsqueeze(1)  # (n,n,d)
print(diff[2,2])    # 这里可见diff[m,m]均为0， diff[i,j]表示特征x_i与特征x_j的差值
sq_dist = torch.sum(diff ** 2, dim=-1)  # (n,n) 计算特征之间的距离
print(sq_dist[2,4])

torch.Size([8, 3]) 
 torch.Size([8, 8, 3])
tensor([0., 0., 0.])
tensor(1.)


In [1]:
import torch
print(torch.rand(1))

tensor([0.8746])


/root/miniconda3/envs/gread/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1记录一下 log核和gauss核之前的写法(no sparse)

In [ ]:
#   def hadamard_product_sparse(self, K, x):
#     # 求解A⊙K(X(t))X(t)
#       num_nodes = K.size(0)  # 节点数量
#       edge_weight = torch.ones(self.edge_index.size(1), device=self.device) # edge_index对应的非零元的值
#       A = torch.sparse_coo_tensor(self.edge_index, edge_weight, (num_nodes, num_nodes)).to(self.device) # torch.Size([2485, 2485])
#       # 获取稀疏矩阵 A 的索引和对应的值
#       indices = A.coalesce().indices()  # 合并重复的索引，并获取稀疏矩阵 A 的索引, torch.Size([2, 10138])
#       values = A.coalesce().values()  # 获取稀疏矩阵 A 的值 torch.Size([10138])
#       # 计算 A 和 K 的 Hadamard 乘积
#       # 只计算 A 中非零元素对应位置的 K 的值
#       hadamard_values = values * K[indices[0], indices[1]]  # torch.Size([10138])
#       # 构建稀疏矩阵的结果
#       AKX = torch_sparse.spmm(self.edge_index, hadamard_values, num_nodes, num_nodes, x)  # 稀疏矩阵和密集矩阵的乘积A⊙K(X(t))X(t) 
#       return AKX  # torch.Size([2485, 64])

In [ ]:
  '''def calculate_log_kernel(self, x):
        n = x.size(0)  # n: 2485
        kernel = torch.zeros((n, n))  # kernel.shape: torch.Size([2485, 2485])

        for i in range(n):
            for j in range(n):
                if i != j:
                    diff = x[i] - x[j]  # x[i]:torch.Size([64])
                    abs_diff = torch.abs(diff)   # abs_diff.shape: torch.Size([64])
                    kernel[i, j] = torch.log(abs_diff.sum())  # 1 范数
        return kernel'''

In [ ]:
# def calculate_log_kernel(self, x):
  #       n = x.size(0)  # 获取样本数量
  #       x_expanded = x.unsqueeze(0).repeat(n, 1, 1)  # 扩展维度并复制，形状为 (n, n, d)
  #       diff = x_expanded - x.unsqueeze(1)  # 计算每对样本之间的差，形状为 (n, n, d)
  #       abs_diff = torch.abs(diff)  # 取绝对值
  #       sum_abs_diff = abs_diff.sum(dim=-1)  # 对最后一个维度求和，得到每对样本的 1 范数差，形状为 (n, n)
  #       s = 1e-4*torch.eye(n).to(x.device)
  #       sum_abs_diff += s  # 加入对角线元素，防止分母为零

  #       # 对角线为零（因为对角线上的元素表示样本自身与自身的差）
  #       log_kernel = torch.zeros_like(sum_abs_diff)  # 创建一个形状相同的零矩阵
  #       nonzero_indices = sum_abs_diff != 0  # 找到非零元素的位置
  #       log_kernel[nonzero_indices] = torch.log(sum_abs_diff[nonzero_indices])  # 对非零元素取对数
  #       return log_kernel
    

In [ ]:
 # def calculate_gauss_kernel(self, x):
  #     n = x.size(0)  # 节点数量
  #     d = x.size(1)  # 特征维度

  #     # 扩展并计算所有节点对之间的差值
  #     x_expanded = x.unsqueeze(0).repeat(n, 1, 1)  # 形状 (n, n, d)
  #     diff = x_expanded - x.unsqueeze(1)  # 形状 (n, n, d)
        
  #     # 计算平方距离矩阵
  #     sq_dist = torch.sum(diff ** 2, dim=-1)  # 形状 (n, n)
      
  #     # 使用向量化计算高斯核矩阵
  #     factor = 1 / ((4 * math.pi * self.epsilon ** 2) ** (d / 2))
  #     exponent = torch.exp(-sq_dist / (4 * self.epsilon ** 2))
  #     kernel = factor * exponent
  #     return kernel

In [ ]:
# elif self.opt['reaction_term'] =='aggdiff-gauss':
    #   # (A - I)X(t) ⊙ K(X(t))X(t)
    #   K = self.calculate_gauss_kernel(x)
    #   AKX = self.hadamard_product_sparse(K, x)  # torch.Size([2485, 64])
    #   reaction = (ax-x)*(AKX)